In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
AAL_pos_path = os.getcwd() + '\\AAL' + '\\Pos'
AAL_neg_path = os.getcwd() + '\\AAL' + '\\Negs'
AAL_abs_path = os.getcwd() + '\\AAL' + '\\Abs'

#Load Pos
filelist = os.listdir(AAL_pos_path)
AAL_pos_df = pd.read_excel(AAL_pos_path + '\\globalVariables.xls',sheetname='0.2')
AAL_pos_df.columns = 'pos ' + 'globalVariables' + '|' + AAL_pos_df.columns
for i in range(len(filelist)):
    if filelist[i] != 'globalVariables.xls':
        temp2 = pd.read_excel(AAL_pos_path + '\\' + filelist[i],sheetname='0.2')
        temp2.columns = ['pos ' + filelist[i][:-4] + '|'] + temp2.columns
        AAL_pos_df = AAL_pos_df.join(temp2)
    else:
        continue
del temp2

#Load Neg
filelist = os.listdir(AAL_neg_path)
AAL_neg_df = pd.read_excel(AAL_neg_path + '\\globalVariables.xls',sheetname='0.2')
AAL_neg_df.columns = 'neg ' + 'globalVariables' + '|' + AAL_neg_df.columns
for i in range(len(filelist)):
    if filelist[i] != 'globalVariables.xls':
        temp2 = pd.read_excel(AAL_neg_path + '\\' + filelist[i],sheetname='0.2')
        temp2.columns = ['neg ' + filelist[i][:-4] + '|'] + temp2.columns
        AAL_neg_df = AAL_neg_df.join(temp2)
    else:
        continue
del temp2

#Load Abs
filelist = os.listdir(AAL_abs_path)
AAL_abs_df = pd.read_excel(AAL_abs_path + '\\globalVariables.xls',sheetname='0.2')
AAL_abs_df.columns = 'abs ' + 'globalVariables' + '|' + AAL_abs_df.columns
for i in range(len(filelist)):
    if filelist[i] != 'globalVariables.xls':
        temp2 = pd.read_excel(AAL_abs_path + '\\' + filelist[i],sheetname='0.2')
        temp2.columns = ['abs ' + filelist[i][:-4] + '|'] + temp2.columns
        AAL_abs_df = AAL_abs_df.join(temp2)
    else:
        continue
del temp2


Vars = pd.read_csv('Variables_Prepd.csv')

In [3]:
def name_parser(s):
    '''
    pass
    '''
    b = s.replace('CorrMatrix_','')
    b = b[:b.find('_rois')]
    return b

def add_meta_data(df):
    '''
    pass
    '''
    temp_df = df.reset_index()
    temp_df['FILE_ID'] = temp_df['index'].apply(name_parser)
    temp_df['SITE_ID'] = temp_df['FILE_ID'].apply(lambda s: ''.join(s.split('_')[:-1]).upper())
    temp_df['subject'] = temp_df['FILE_ID'].apply(lambda s: s[s.find('_00')+3:])
    temp_df.set_index('index',inplace=True)
    return temp_df
    

In [4]:
AAL_pos_df = add_meta_data(AAL_pos_df)
AAL_neg_df = add_meta_data(AAL_neg_df)
AAL_abs_df = add_meta_data(AAL_abs_df)

#Get rid of subjects with missing data in their correlation matrix
bad_aal = ['CMU_b_0050651', 'CMU_b_0050657', 'CMU_b_0050669', 'Caltech_0051457', 'Caltech_0051458', 
'Caltech_0051459', 'Caltech_0051460', 'Caltech_0051464', 'Caltech_0051466', 'Caltech_0051467', 
'Caltech_0051468', 'Caltech_0051469', 'Caltech_0051471', 'Caltech_0051473', 'Caltech_0051478', 'Caltech_0051483', 
'Caltech_0051485', 'Caltech_0051487', 'Caltech_0051489', 'Caltech_0051490', 'KKI_0050790', 'Leuven_2_0050727', 
'MaxMun_a_0051364', 'MaxMun_c_0051336', 'NYU_0051020', 'NYU_0051033', 'NYU_0051036', 'NYU_0051100', 'NYU_0051118', 
'Pitt_0050004', 'Pitt_0050007', 'Pitt_0050011', 'Pitt_0050016', 'Pitt_0050024', 'Pitt_0050032', 'Pitt_0050035', 
'Pitt_0050037', 'Pitt_0050041', 'Pitt_0050045', 'Pitt_0050050', 'Pitt_0050052', 'Pitt_0050060', 'SBL_0051558', 
'SBL_0051560', 'SBL_0051562', 'SBL_0051564', 'SBL_0051565', 'SBL_0051566', 'SBL_0051567', 'SBL_0051571', 
'SBL_0051572', 'SBL_0051573', 'SBL_0051577', 'SBL_0051578', 'SBL_0051579', 'SBL_0051580', 'SBL_0051582', 
'SBL_0051583', 'SBL_0051584', 'SBL_0051585', 'SDSU_0050182', 'SDSU_0050184', 'SDSU_0050195', 'SDSU_0050209', 
'SDSU_0050210', 'SDSU_0050216', 'Trinity_0050259']

AAL_pos_df = AAL_pos_df[~AAL_pos_df.FILE_ID.isin(bad_aal)]
AAL_neg_df = AAL_neg_df[~AAL_neg_df.FILE_ID.isin(bad_aal)]
AAL_abs_df = AAL_abs_df[~AAL_abs_df.FILE_ID.isin(bad_aal)]

#Add usefull demographic data to the dataframes
AAL_pos_df = pd.merge(AAL_pos_df,Vars[['FILE_ID','DX_GROUP','SEX', 'AGE_AT_SCAN']],on='FILE_ID')
AAL_neg_df = pd.merge(AAL_neg_df,Vars[['FILE_ID','DX_GROUP','SEX', 'AGE_AT_SCAN']],on='FILE_ID')
AAL_abs_df = pd.merge(AAL_abs_df,Vars[['FILE_ID','DX_GROUP','SEX', 'AGE_AT_SCAN']],on='FILE_ID')

In [5]:
def select_subjects(df, sites=None, sex=None, age_range=None, group=None):
    '''
    pass
    '''
    
    dictionary = d = {'m': 1, 'f': 2, 'ASD': 1, 'HC': 2} #dict for transforming the inputs to values found in the df
    #Fill the variables in such a way that it selects all subjects
    if sites == None: 
        sites = list(df.SITE_ID.unique())
    if sex == None:
        sex = ['m','f']
    if age_range == None:
        age_range = (0,150)
    if group == None:
        group = ['HC','ASD']
    
    #Actual code
    if isinstance(group, list) == False:
        group = group.split()
    sex = list(map(dictionary.get,sex))
    group = list(map(dictionary.get,group))
    
    return df[(df['SITE_ID'].isin(sites)) & 
              (df['SEX'].isin(sex)) & 
              (df['AGE_AT_SCAN'].between(age_range[0], age_range[1])) & 
              (df['DX_GROUP'].isin(group)) 
             ]

## Kfold CV

** Pos **

In [19]:
data = select_subjects(AAL_pos_df,sex='m',sites=['YALE','PITT','NYU','TRINITY','STANFORD','UCLA1','SDSU','UM1'])
X = data.drop(['DX_GROUP','SEX','AGE_AT_SCAN','FILE_ID','SITE_ID','subject'],axis=1)
y = data['DX_GROUP']

In [7]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=10,random_state=101)

In [22]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

svc_rbf = make_pipeline(StandardScaler() ,SVC(random_state = 101,C=10,gamma='auto'))

sfs = SFS(svc_rbf, 
           k_features=(1,30), 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=cv,n_jobs=-1)

sfs = sfs.fit(X, y)

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 639 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 817 out of 817 | elapsed:   14.9s finished

[2018-09-20 15:07:58] Features: 1/30 -- score: 0.5628846153846154[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 639 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 801 out of 816 | elapsed:   14.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 816 out of 816 | elapsed:   14.9s finished

[2018-09-20 15:08:13] Features: 2/30 -- score: 0.5912179487179487[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 231 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 637 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 815 out of 815 | elapsed:   15.3s finished

[2018-09-20 15:

[2018-09-20 15:15:04] Features: 22/30 -- score: 0.6955128205128205[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 795 out of 795 | elapsed:   22.3s finished

[2018-09-20 15:15:27] Features: 23/30 -- score: 0.7008333333333334[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 794 out of 794 | elapsed:   22.4s finished

[2018-09-20 15:15:50] Features: 24/30 -- score: 0.6956410256410256[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 349 tasks 

** Neg **

In [23]:
data = select_subjects(AAL_neg_df,sex='m',sites=['YALE','PITT','NYU','TRINITY','STANFORD','UCLA1','SDSU','UM1'])
X = data.drop(['DX_GROUP','SEX','AGE_AT_SCAN','FILE_ID','SITE_ID','subject'],axis=1)
y = data['DX_GROUP']

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

svc_rbf = make_pipeline(StandardScaler() ,SVC(random_state = 101,C=10,gamma='auto'))

sfs2 = SFS(svc_rbf, 
           k_features=(1,30), 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=cv,n_jobs=-1)

sfs2 = sfs2.fit(X, y)

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 231 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 637 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 817 out of 817 | elapsed:   15.0s finished

[2018-09-20 15:25:19] Features: 1/30 -- score: 0.5784615384615386[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 639 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 801 out of 816 | elapsed:   14.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 816 out of 816 | elapsed:   14.8s finished

[2018-09-20 15:25:34] Features: 2/30 -- score: 0.591346153846154[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 229 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 635 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 815 out of 815 | elapsed:   15.6s finished

[2018-09-20 15:2

** Concat **

In [38]:
temp = AAL_neg_df.merge(AAL_pos_df)
data = select_subjects(temp,sex='m',sites=['YALE','PITT','NYU','TRINITY','STANFORD','UCLA1','SDSU','UM1'])
X = data.drop(['DX_GROUP','SEX','AGE_AT_SCAN','FILE_ID','SITE_ID','subject'],axis=1)
y = data['DX_GROUP']

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

svc_rbf = make_pipeline(StandardScaler() ,SVC(random_state = 101,C=10,gamma='auto'))

sfs3 = SFS(svc_rbf, 
           k_features=(1,30), 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=cv,n_jobs=-1)

sfs3 = sfs3.fit(X, y)
del temp

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 230 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 636 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 1634 out of 1634 | elapsed:   23.6s finished

[2018-09-20 15:44:47] Features: 1/30 -- score: 0.5784615384615386[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 638 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 1204 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 1618 out of 1633 | elapsed:   22.8s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 1633 out of 1633 | elapsed:   22.9s finished

[2018-09-20 15:45:10] Features: 2/30 -- score: 0.6013461538461539[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 229 tasks      | elapsed:    8.6s
[Parallel(n_jobs=

[Parallel(n_jobs=-1)]: Done 1618 out of 1618 | elapsed:   34.8s finished

[2018-09-20 15:53:14] Features: 17/30 -- score: 0.7110897435897436[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done 1617 out of 1617 | elapsed:   35.5s finished

[2018-09-20 15:53:50] Features: 18/30 -- score: 0.7163461538461539[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 1442 task

** Abs **

In [ ]:
data = select_subjects(AAL_abs_df,sex='m',sites=['YALE','PITT','NYU','TRINITY','STANFORD','UCLA1','SDSU','UM1'])
X = data.drop(['DX_GROUP','SEX','AGE_AT_SCAN','FILE_ID','SITE_ID','subject'],axis=1)
y = data['DX_GROUP']

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

svc_rbf = make_pipeline(StandardScaler() ,SVC(random_state = 101,C=10,gamma='auto'))

sfs4 = SFS(svc_rbf, 
           k_features=(1,30), 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=cv,n_jobs=-1)

sfs4 = sfs4.fit(X, y)

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 227 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 633 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 817 out of 817 | elapsed:   17.8s finished

[2018-09-24 12:46:35] Features: 1/30 -- score: 0.5628846153846154[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 230 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 636 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 816 out of 816 | elapsed:   17.6s finished

[2018-09-24 12:46:53] Features: 2/30 -- score: 0.5912179487179487[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 222 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 628 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 800 out of 815 | elapsed:   16.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 815 out of 815 | elapsed:   16.6s finished

[2018-09-24 12: